In [6]:
import pandas as pd
from glob import glob
import numpy as np
from tools.procOps import *
from tools.fileOps import *
import itertools
import multiprocessing
from scipy.special import beta
from tools.bio import *

In [289]:
sun_df = pd.read_csv('copy_number/hg38_features.txt', sep='\t')
sun_df.columns = ['loc', 'NAB', 'NC', 'ND', 'NN']
sun_df['NAB'] = [1 if x > 0 else 0 for x in sun_df['NAB']]
sun_df['NC'] = [1 if x > 0 else 0 for x in sun_df['NC']]
sun_df['ND'] = [1 if x > 0 else 0 for x in sun_df['ND']]
sun_df['NN'] = [1 if x > 0 else 0 for x in sun_df['NN']]
sun_df.head()

,loc,NAB,NC,ND,NN
0,56,0,0,0,1
1,178,0,0,1,0
2,240,0,0,0,1
3,386,1,0,0,0
4,393,1,0,0,0


In [290]:
# remove bad positions
bad_positions = set([48646, 75180, 17731, 43539, 2967, 54815, 64033, 89900, 66088, 68137, 90156, 85939, 87309, 70198, 54452, 1594, 77455, 8253, 73792, 7776, 60995, 44613, 96353, 24138, 48718, 13391, 47185, 24659, 29269, 13913, 23133, 68191, 29280, 69729, 1635, 88677, 10513, 74860, 54893, 90226, 57973, 74358, 58487, 82556, 10517, 76419, 59524, 74373, 41094, 8843, 63630, 74383, 5267, 57493, 12439, 35992, 85862, 3235, 48297, 68722, 24754, 74420, 95925, 7356, 7871, 38598, 18209, 5841, 39122, 67796, 68309, 94430, 6370, 93478, 49894, 37095, 52457, 16620, 92397, 60142, 76015, 41712, 93480, 41716, 54006, 22990, 48376, 16597, 92418, 40711, 93484, 63242, 26380, 898, 92431, 29144, 49941, 43799, 48409, 91419, 90398, 92448, 25889, 64803, 10534, 24359, 39720, 46889, 93482, 25900, 93486, 76080, 20276, 78646, 53056, 86339, 76104, 41186, 14670, 67410, 23382, 83803, 17245, 17760, 63844, 88806, 74086, 26983, 46953, 88039, 878, 12655, 16345, 7957, 38774, 50409, 386, 1496, 41878, 66952, 393, 53655, 60815, 15250, 58691, 41454, 64406, 70039, 2459, 11679, 86432, 34203, 94628, 2469, 72614, 84903, 74664, 42921, 29100, 40370, 37299, 78266, 9149, 61385, 65996, 29133, 85454, 89553, 45522, 21459, 27096, 47065, 51164, 76769, 8674, 85806, 56292, 88037, 89169, 47080, 54766, 30196, 70649, 70651, 31230, 92159])
sun_df = sun_df[~sun_df['loc'].isin(bad_positions)]

In [18]:
# load parsed pileups
files = glob('/hive/users/ifiddes/simons_normals/96kb_consensus/*.parsed_pileup.txt')
dfs = {}
for f in files:
    n = os.path.basename(f).split('.')[0]
    dfs[n] = pd.read_csv(f, sep='\t', index_col=0)

In [19]:
# load C/D copy number estimates

files = glob('/hive/users/ifiddes/simons_normals/*.filtered.txt')

from collections import Counter
def convert(x):
    x = x.split(':')
    n, v = x
    v = int(v)
    return n, v

copy_number = {}
for x in files:
    n = os.path.basename(x).split('.')[0]
    l = open(x).next().rstrip().split()
    c = []
    for x in l[2:4]:
        _, v = convert(x)
        c.append(v)
    copy_number[n] = c

In [96]:
# filter dataframes for C = 2 and D = 2
# also filter for informative positions
filtered_dfs = {}
for n, df in dfs.iteritems():
    c = copy_number[n]
    if sum(c) != 4:
        continue
    df_m = df.merge(sun_df, on='loc')
    df_m = df_m[df_m['loc'].isin(sun_df['loc'])]
    filtered_dfs[n] = df_m

In [295]:
useful_positions = set.intersection(*[set(x['loc']) for x in filtered_dfs.itervalues()])
filtered_sun_df = sun_df[sun_df['loc'].isin(useful_positions)]

In [235]:
# k is a vector for each position in the suns that tell us how many genomes contain the paratype
# since we are at 4 2 2 2 for everyone it is 4 or 2 * number of genomes
# define an arbitrary cutoff of 0.01 ratio and 10 coverage to define a missing site
# k = count of the number of paratypes with alt
# i = count of the number of paratypes with ref
k = defaultdict(dict)
l = defaultdict(dict)
actual_alt = defaultdict(dict)
actual_ref = defaultdict(dict)
for n, df_m in filtered_dfs.iteritems():
    for _, s in df_m.iterrows():
        if s.coverage >= 10 and s.ratio >= 0.01:
            num_alt = 4 * s.NAB + 2 * s.NC + 2 * s.ND + 2 * s.NN  # number of alt paratypes
            num_ref = 10 - num_alt  # valid because these are all 4-2-2-2
            k[n][s['loc']] = num_alt
            l[n][s['loc']] = num_ref
            actual_alt[n][s['loc']] = s.alt_count
            actual_ref[n][s['loc']] = s.ref_count

In [ ]:
for n, df_m in filtered_dfs.iteritems():
    for _, s in df_m.iterrows():
        if s['loc'] != 8285:
            continue
        if s.coverage >= 10 and s.ratio >= 0.01:
            print s.alt_count, s.ref_count

In [236]:
# all proposed genotypes
def construct_C(max_n=12, min_n=8, k=4):
    list1=np.arange(0,6)
    r = []
    for i in itertools.product(list1,repeat=k):
        if min_n <= np.sum(i) <= max_n:
            r.append(i)
    return np.array(r)

C = construct_C()

In [297]:
# number of haplotypes that are expected to be alt in each C
ft = filtered_sun_df.set_index('loc').T
alt_options = np.dot(C, ft)

# ref options -- first construct vector of total number of paratypes in each row of C
tot = C.sum(axis=1)
tot = np.array([tot] * alt_options.shape[1]).T # turn into total matrix
ref_options = tot - alt_options  # remove alt to get ref counts

In [313]:
#precompute tot_i
tot = []
for loc in filtered_sun_df['loc']:
    tot.append(sum(actual_alt[g][loc] + actual_ref[g][loc] for g in actual_alt if loc in actual_alt[g]))


all_a = []
all_b = []
for c, (alt_i, ref_i) in enumerate(zip(alt_options, ref_options)):
    a = []
    b = []
    for i, (_, s) in enumerate(filtered_sun_df.iterrows()):
        try:
            scaled_alt_i = sum(1.0 * k[g][s['loc']] / alt_i[i] * actual_alt[g][s['loc']] for g in actual_alt if s['loc'] in k[g])
        except ZeroDivisionError:
            scaled_alt_i = 0
        try:
            scaled_ref_i = sum(1.0 * l[g][s['loc']] / ref_i[i] * actual_ref[g][s['loc']] for g in actual_ref if s['loc'] in l[g])
        except ZeroDivisionError:
            scaled_ref_i = 0
        u = scaled_alt_i / (scaled_alt_i + scaled_ref_i)
        a.append(u * (tot[i] - 1))
        b.append((1 - u) * (tot[i] - 1))
    all_a.append(a)
    all_b.append(b)
    if c != 0 and c % 10 == 0:
        print c

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390


KeyboardInterrupt: 

In [239]:
for g in actual_ref:
    if s['loc'] not in l[g]:
        continue
    q = 1.0 * l[g][s['loc']] / ref_i[i] * actual_ref[g][s['loc']]

ZeroDivisionError: float division by zero

In [78]:
# try a single random genome
n = 'LP6005441-DNA_G04'
df_m = filtered_dfs[n]

In [ ]:
alt_vector = []
ref_vector = []
for _, s in df_m.iterrows():
    a = sum(1.0 * k[n][s['loc']] / s.)

In [62]:
def calculate_adjusted_alt(s):
    return 1.0 * sum()
    sum(k)
    return 1.0 * s.alt_count * k[s['loc']] / sum(k.values())

def calculate_adjusted_ref(s):
    return 1.0 * s.ref_count * i[s['loc']] / sum(i.values())


df_m['adjusted_alt'] = df_m.apply(calculate_adjusted_alt, axis=1)
df_m['adjusted_ref'] = df_m.apply(calculate_adjusted_ref, axis=1)

In [108]:
k = defaultdict(int)
l = defaultdict(int)
for n, df_m in filtered_dfs.iteritems():
    for _, s in df_m.iterrows():
        if s.coverage >= 10 and s.ratio >= 0.01:
            num_alt = 4 * s.NAB + 2 * s.NC + 2 * s.ND + 2 * s.NN  # number of alt paratypes
            num_ref = 10 - num_alt  # valid because these are all 4-2-2-2
            k[s['loc']] += num_alt
            l[s['loc']] += num_ref

sun_df['k'] = [k[x] for x in sun_df['loc']]
sun_df['l'] = [l[x] for x in sun_df['loc']]

In [117]:
# calculate actualAlt and actualRef from training data
actual_alt = defaultdict(float)
actual_ref = defaultdict(float)
for n, df_m in filtered_dfs.iteritems():
    for _, s in df_m.iterrows():
        if s.coverage >= 10 and s.ratio >= 0.01:
            actual_alt[s['loc']] += s.alt_count
            actual_ref[s['loc']] += s.ref_count
            
sun_df['actual_alt'] = [actual_alt[x] for x in sun_df['loc']]
sun_df['actual_ref'] = [actual_ref[x] for x in sun_df['loc']]

In [110]:
def construct_C(max_n=12, min_n=8, k=4):
    list1=np.arange(0,6)
    r = []
    for l in itertools.product(list1,repeat=k):
        if min_n <= np.sum(l) <= max_n:
            r.append(l)
    return np.array(r)

C = construct_C()